In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time

%matplotlib tk

x = np.arange(0, 1023)
y = np.random.randint(-32000, 32001, 1023)

fig, ax = plt.subplots()
line, = ax.plot(x, y)
ax.set_ylim(-32000, 32000)
ax.set_xlim(0, 1023)

tstart = time.time()
num_plots = 0
while time.time() - tstart < 1:
    line.set_ydata(np.random.randint(-32000, 32001, 1023))
    fig.canvas.draw()
    fig.canvas.flush_events()
    num_plots += 1

print(num_plots)

36


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import serial
import time
from tkinter import TclError

# für JupyterLab (Widget-Backend)
%matplotlib tk

# Konstanten
CHUNK = 1024          # kleinere Puffergröße, schnellere Updates
SERIAL_PORT = '/dev/ttyUSB0'  # evtl. anpassen
BAUDRATE = 1000000

# Serielle Verbindung öffnen
ser = serial.Serial(SERIAL_PORT, BAUDRATE)

# Matplotlib-Plot vorbereiten
fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(0, CHUNK)
data_np = np.zeros(CHUNK)
line, = ax.plot(x, data_np, '-', lw=2)
ax.set_title('Live Serial Data')
ax.set_xlabel('Samples')
ax.set_ylabel('Amplitude')
ax.set_ylim(-10000, 10000)
ax.set_xlim(0, CHUNK)
plt.show()

print('Stream gestartet')

# Frame-Rate messen
frame_count = 0
start_time = time.time()

# Hilfspuffer
index = 0

try:
    while True:
        # Zeile lesen
        raw_line = ser.readline().decode('utf-8').strip()
        
        if raw_line:
            try:
                value = float(raw_line.split(',')[0])  # Nur ersten Wert verwenden
                data_np[index] = value
                index += 1

                if index >= CHUNK:
                    # Update Plot
                    line.set_ydata(data_np)
                    fig.canvas.draw_idle()
                    fig.canvas.flush_events()
                    
                    index = 0  # Reset
                    frame_count += 1

            except ValueError:
                pass

except (KeyboardInterrupt, TclError):
    frame_rate = frame_count / (time.time() - start_time)
    print('\nStream gestoppt')
    print('Durchschnittliche Framerate: {:.0f} FPS'.format(frame_rate))

finally:
    ser.close()

Stream gestartet
